In [4]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [10]:
sndata = pd.read_csv('lcparam_full_long.txt', sep=' ', skiprows=0)
z = sndata['zcmb'].to_numpy()
mb = sndata['mb'].to_numpy()

In [27]:
cov_flatten = np.loadtxt('sys_full_long.txt', delimiter=' ')
cov_T = []
for i in range(1048):
    cov_row = cov_flatten[i*1048:(i+1)*1048]
    cov_T.append(cov_row)
cov = np.array(cov_T)

In [ ]:
def cholesky_arr(cov):
    if 
    L = np.zeros_like(cov)
    for i in range(cov.shape[0]):
        for j in range(i+1):
            if i == j:
                L[i][j] = np.sqrt(cov[j][j] - np.sum(L[j][:j]**2))
            else:
                L[i][j] = (cov[i][j] - np.sum(L[i][:j]*L[j][:j])) / L[j][j]
    return L

def inv_cholesky(L):


In [57]:
Arr = np.array([[5.,3.],[3.,5.]])
cholesky_arr(Arr)

array([[2.23606798, 0.        ],
       [1.34164079, 1.78885438]])

In [58]:
from numpy.linalg import eig, cholesky
eig(Arr)
cholesky(Arr)

array([[2.23606798, 0.        ],
       [1.34164079, 1.78885438]])